In [23]:
import dask.dataframe as dd
import pandas as pd

pd.set_option('future.no_silent_downcasting', True)
pd.options.display.max_columns = 
pd.set_option('display.precision', 2)

In [24]:
path_mat = "Data/Predict_student_performance/student-mat.csv"       # path might not work when the file is executed directly
path_por = "Data/Predict_student_performance/student-por.csv"       # if direct execute is needed add "../" before Data

In [25]:
df_mat = dd.read_csv(path=path_mat, sep=";")
#df_mat.compute()

In [26]:
df_por = dd.read_csv(path=path_por, sep=";")
#df_por.compute()

In [27]:
dfs = [df_mat, df_por]

In [28]:
# df_mat.columns

Adjust the following: <br>

- address: U: 0, R: 1
- famsize: LE3: 0, GT3: 1
- Pstatus: T: 0, A: 1

In [29]:
keep_columns=['sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'activities', 
       'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3']
conv_columns=['sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'activities', 'romantic']
replace_dict={'F': 0, 'M': 1, 'U': 0, 'R': 1, 'LE3': 0, 'GT3': 1, 'T': 0, 'A': 1, 'yes': 1, 'no': 0}
transformed_dfs = []                                      # List to store transf. data frames

In [30]:
def transform_dfs (dfs, keep_columns, replace_dict, conv_columns):
    
    for df in dfs:
        df = df[keep_columns]                               # selecting the only columns necessary 
        df = df.replace(replace_dict)                       # replace genders and higher edu values with 0s and 1s
        df = df.dropna()

        for column in conv_columns:
            df[column] = df[column].astype('bool')          # change dtype to boolean

        # print(df.compute())
        # print(df.dtypes)

        transformed_dfs.append(df.compute())                # build list of transformed data frames
    
    return transformed_dfs

In [31]:
transform_dfs(dfs=dfs, keep_columns=keep_columns, replace_dict=replace_dict, conv_columns=conv_columns)

[       sex  address  famsize  Pstatus  Medu  Fedu  traveltime  studytime  \
 0    False    False     True     True     4     4           2          2   
 1    False    False     True    False     1     1           1          2   
 2    False    False    False    False     1     1           1          2   
 3    False    False     True    False     4     2           1          3   
 4    False    False     True    False     3     3           1          2   
 ..     ...      ...      ...      ...   ...   ...         ...        ...   
 390   True    False    False     True     2     2           1          2   
 391   True    False    False    False     3     1           2          1   
 392   True     True     True    False     1     1           1          1   
 393   True     True    False    False     3     2           3          1   
 394   True    False    False    False     1     1           1          1   
 
      failures schoolsup famsup activities nursery higher romantic  famrel

In [32]:
# concatenate data frames
student_perf = dd.concat(transformed_dfs)
# student_perf.compute()

In [33]:
# normalize Grades and calculate AVG and drop Grade columns
grades = ['G1', 'G2', 'G3']

for grade in grades:
    student_perf[grade] = (student_perf[grade] - 0) / (20 - 0)

student_perf['AVG_G'] = student_perf[grades].mean(axis=1)
student_perf = student_perf.drop(grades, axis=1)
# student_perf.compute()

In [37]:
# student_perf.compute().describe()

,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,AVG_G
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000
mean,2.603448,2.387931,1.522989,1.970307,0.264368,3.935824,3.201149,3.156130,1.494253,2.284483,3.543103,4.434866,0.563362
std,1.124907,1.099938,0.731727,0.834353,0.656142,0.933401,1.031507,1.152575,0.911714,1.285105,1.424703,6.210017,0.160940
min,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.066667
25%,2.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,0.466667
50%,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,0.566667
75%,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,6.000000,0.666667
max,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,0.966667


In [39]:
# add column to address if failed or not - threshold 60 % of AVG_G

student_perf['passed'] = student_perf['AVG_G'] >= 0.6

# student_perf.compute()

,sex,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,AVG_G,passed
0,False,False,True,True,4,4,2,2,0,yes,no,no,yes,yes,no,4,3,4,1,1,3,6,0.283333,False
1,False,False,True,False,1,1,1,2,0,no,yes,no,no,yes,no,5,3,3,1,1,3,4,0.266667,False
2,False,False,False,False,1,1,1,2,3,yes,no,no,yes,yes,no,4,3,2,2,3,3,10,0.416667,False
3,False,False,True,False,4,2,1,3,0,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,0.733333,True
4,False,False,True,False,3,3,1,2,0,no,yes,no,yes,yes,no,4,3,2,1,2,5,4,0.433333,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,False,True,True,False,2,3,1,3,1,no,no,yes,no,yes,no,5,4,2,1,2,5,4,0.516667,False
645,False,False,False,False,3,1,1,2,0,no,yes,no,yes,yes,no,4,3,4,1,1,1,4,0.766667,True
646,False,False,True,False,1,1,2,2,0,no,no,yes,yes,yes,no,1,1,1,1,1,5,6,0.533333,False
647,True,False,False,False,3,1,2,1,0,no,no,no,no,yes,no,2,4,5,3,4,2,6,0.500000,False
